# Using the predictions

We now have a .parquet with predictions for every hour for a year for each LocationID

We will normalise these predictions and store these normalised predictions in a new parquet file

We may also join with coordinations for each LocationID, though this may not be strictly neccessary at this stage

In [1]:
import pandas as pd

predictions = pd.read_parquet('predictions_datetime.parquet')

In [2]:
predictions

,location_id,hour,day,month,week_of_year,prediction,datetime
0,4,0,1,7,30,58,2023-07-25 00:00:00
1,12,0,1,7,30,1,2023-07-25 00:00:00
2,13,0,1,7,30,39,2023-07-25 00:00:00
3,24,0,1,7,30,63,2023-07-25 00:00:00
4,41,0,1,7,30,155,2023-07-25 00:00:00
...,...,...,...,...,...,...,...
579805,246,0,3,7,30,327,2024-07-25 00:00:00
579806,249,0,3,7,30,1072,2024-07-25 00:00:00
579807,261,0,3,7,30,233,2024-07-25 00:00:00
579808,262,0,3,7,30,99,2024-07-25 00:00:00


In [3]:
# # Normalize the predictions using Min-Max scaling
# predictions['normalised_prediction'] = (predictions['prediction'] - predictions['prediction'].min()) / (predictions['prediction'].max() - predictions['prediction'].min())
# predictions

In [4]:
# import pandas as pd

# predictions = pd.read_parquet('predictions.parquet')

# # Extract the minimum and maximum values
# prediction_min = predictions['prediction'].min()
# prediction_max = predictions['prediction'].max()

# # Store the min and max values in variables for use in another Jupyter Notebook
# min_value = prediction_min.item()
# max_value = prediction_max.item()

# # Print the normalized predictions
# predictions['normalized_prediction'] = (predictions['prediction'] - min_value) / (max_value - min_value)
# print(predictions)

# # Store the min and max values in a separate DataFrame
# min_max_values = pd.DataFrame({'min_value': [min_value], 'max_value': [max_value]})

# # Save the min and max values to a file
# min_max_values.to_csv('min_max_values.csv', index=False)


In [5]:
# min_value

In [6]:
# # Drop the 'prediction' column from the predictions DataFrame
# predictions = predictions.drop('prediction', axis=1)
# predictions

In [7]:
# import pyarrow.parquet as pq
# import pyarrow as pa

# # Convert DataFrame to PyArrow Table
# table = pa.Table.from_pandas(predictions)

# # Write the Parquet file
# pq.write_table(table, 'normalised_predictions.parquet')

## Connecting to database

In [8]:
# ensure datetime is stored as datetime in the db
predictions['datetime'] = pd.to_datetime(predictions['datetime'])

In [9]:
predictions.dtypes

location_id              int64
hour                     int64
day                      int64
month                    int64
week_of_year             int64
prediction               int64
datetime        datetime64[ns]
dtype: object

In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Assuming you already have the predictions DataFrame

# Set up the PostgreSQL connection
database = 'cafes_manhattan'
user = 'cormacegan'
password = ''
host = 'localhost'
port = '5432'

# Create the engine for connecting to the PostgreSQL server
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Store the DataFrame as a table in the PostgreSQL server
table_name = 'predictions'
predictions.to_sql(table_name, engine, if_exists='replace', index=False)

# Close the database connection
engine.dispose()
